# Setting Up Libraries

In [2]:
import numpy as np
np.random.seed(42)

In [3]:
import tensorflow as tf
tf.set_random_seed(42)

In [4]:
from keras.models import Sequential,load_model

Using TensorFlow backend.


In [5]:
from keras.layers import Dense,Activation

In [6]:
from keras.layers import LSTM,Dropout

In [7]:
from keras.layers import TimeDistributed

In [8]:
from keras.layers.core import Dense, Activation, Dropout, RepeatVector

In [9]:
from keras.optimizers import RMSprop

In [10]:
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 12, 5